Import des librairies

In [29]:
#!pip install openmeteo_requests
#!pip install requests_cache retry_requests tqdm pandas
#!pip install pulp

In [2]:
import pandas as pd
import numpy as np
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Entrainement du modèle de machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#Visualisation des données
import plotly.express as px
import plotly.graph_objects as go

import pulp as pl # Pour la programmation linéaire

Import CSV

In [3]:
Lien = r'Projet_ML.csv'
data = pd.read_csv(Lien, sep=";",decimal='.')
data.head()

,Date,P_Chaud,P_Froid,T_Ext,Hum_Ext,Ensoleillement,V_Vent
0,01/01/2023 00:00,18.274564,4.453890,16.04814645,45.579264,-3.206792,17.767377
1,01/01/2023 01:00,16.869887,4.554443,15.63389643,48.694887,-2.675742,19.335602
2,01/01/2023 02:00,21.838088,5.000000,15.36290866,52.479784,-2.950760,19.249763
3,01/01/2023 03:00,28.072719,4.824986,15.18215987,55.898372,-3.503033,16.579626
4,01/01/2023 04:00,28.272941,4.758345,14.92442452,59.608522,-3.461036,15.723777


In [4]:
#Nettoyage des données avec [-11059] No Good Data For Calculation
data = data.replace('[-11059] No Good Data For Calculation', np.nan)

# Conversion des colonnes en types appropriés
data['T_Ext'] = data['T_Ext'].astype(float)

# Conversion de la colonne 'Date' en type datetime
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y %H:%M', errors='raise')
data['Date'] = data['Date'].dt.tz_localize('UTC')
print(data.dtypes)
print(data.head())

Date              datetime64[ns, UTC]
P_Chaud                       float64
P_Froid                       float64
T_Ext                         float64
Hum_Ext                       float64
Ensoleillement                float64
V_Vent                        float64
dtype: object
                       Date    P_Chaud   P_Froid      T_Ext    Hum_Ext  \
0 2023-01-01 00:00:00+00:00  18.274564  4.453890  16.048146  45.579264   
1 2023-01-01 01:00:00+00:00  16.869887  4.554443  15.633896  48.694887   
2 2023-01-01 02:00:00+00:00  21.838088  5.000000  15.362909  52.479784   
3 2023-01-01 03:00:00+00:00  28.072719  4.824986  15.182160  55.898372   
4 2023-01-01 04:00:00+00:00  28.272941  4.758345  14.924425  59.608522   

   Ensoleillement     V_Vent  
0       -3.206792  17.767377  
1       -2.675742  19.335602  
2       -2.950760  19.249763  
3       -3.503033  16.579626  
4       -3.461036  15.723777  


Données Météo

In [5]:
# Mise en place du client API Open-Meteo avec cache et réessai en cas d'erreur
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Assurez-vous que toutes les variables météorologiques requises sont répertoriées ici
# L'ordre des variables dans hourly ou daily est important pour les assigner correctement ci-dessous

url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 48.897962, #Coordonnées Site
	"longitude": 2.233007,
	"hourly": ["temperature_2m", "relative_humidity_2m", "rain"],
}
responses = openmeteo.weather_api(url, params=params)

#Premier emplacement de traitement. Ajouter une boucle for pour plusieurs emplacements ou modèles météorologiques
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Traitement des données horaires. L'ordre des variables doit être le même que celui demandé.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["rain"] = hourly_rain

Meteo_pred = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", Meteo_pred)

Coordinates: 48.900001525878906°N 2.239999771118164°E
Elevation: 49.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                          date  temperature_2m  relative_humidity_2m  rain
0   2025-10-20 00:00:00+00:00       15.669499                  89.0   0.0
1   2025-10-20 01:00:00+00:00       15.969500                  86.0   0.0
2   2025-10-20 02:00:00+00:00       15.669499                  86.0   0.0
3   2025-10-20 03:00:00+00:00       15.219500                  87.0   0.0
4   2025-10-20 04:00:00+00:00       14.669499                  84.0   0.0
..                        ...             ...                   ...   ...
163 2025-10-26 19:00:00+00:00        6.658500                  88.0   0.0
164 2025-10-26 20:00:00+00:00        5.958500                  91.0   0.0
165 2025-10-26 21:00:00+00:00        5.358500                  93.0   0.0
166 2025-10-26 22:00:00+00:00        4.808500                  94.0   0.0
167 2025-10-26 23:00:00+00:00        4.308500                  

In [6]:
# fusionner les données température et relative humidity avec les données principales
Meteo_pred.rename(columns={"date": "Date", "temperature_2m": "T_Ext", "relative_humidity_2m": "HR_Ext"}, inplace=True)
# print(Meteo_pred.dtypes)

Questions : 
Quel est la plage horaire de fonctionnement des machines ? (nombres d'heure min avant coupure de la machine)
Ajout de colonnes pour le fonctionnement de chaque machine. Sur le total combien a t-elle produit. Quelle était son COP ?
Est ce qu'une autre machine aurait pu faire mieux ?
Ordre de priorité des machines ? hiver -> AP, biofioul ? demi saison TFP ? Prevoir une nouvelle tfp moins puissance ?
Comment prendre en compte la typologie des batiments ? Prendre la puissance souscrite ?
Faire une premiere prédiction a partir des données actuelles avec la température et l'humidité.
Prévoir l'éloignement de la sst ? Complexe car réseau non fiable peut etre ? Possibilité de prendre en compte le DP de chaque SST ?



In [7]:
#visualisation des données puissance

figure = go.Figure()
figure.add_trace(go.Scatter(x=data['Date'], y=data['P_Chaud'], mode='lines', name='Puissance Totale Chaud (MW)', line=dict(color='red')))
figure.add_trace(go.Scatter(x=data['Date'], y=data['P_Froid'], mode='lines', name='Puissance Froid (MW)', line=dict(color='blue')))
figure.update_layout(title='Puissance Chaud et Froid au cours du temps', xaxis_title='Date', yaxis_title='Puissance (MW)')
figure.show()

### 🧠 Introduction

Avant l'entraînement du modèle, il est essentiel d'ajouter des variables qui capturent la **dynamique temporelle** du réseau énergétique.  
Les deux indicateurs suivants permettent d'introduire une **mémoire du passé** :

- `Puissance_lag1` : valeur de la puissance à l'heure précédente (inertie horaire).  
- `Puissance_roll6h` : moyenne glissante sur les 6 dernières heures (tendance récente).

Ces nouvelles variables améliorent la capacité du modèle à prédire la consommation ou la production, car elles reflètent le comportement temporel réel du réseau.


In [8]:
# --- Feature engineering ---
data['Hour'] = data['Date'].dt.hour
data['Day'] = data['Date'].dt.day
data['Month'] = data['Date'].dt.month
data['Year'] = data['Date'].dt.year
data['Weekend'] = (data['Date'].dt.weekday >= 5).astype(int)

# Lags et moyennes glissantes
data['Puissance_Chaud_lag1'] = data['P_Chaud'].shift(1)
data['Puissance_Chaud_roll6h'] = data['P_Chaud'].rolling(6).mean()
data['Puissance_Froid_lag1'] = data['P_Froid'].shift(1)
data['Puissance_Froid_roll6h'] = data['P_Froid'].rolling(6).mean()
# Suppression des valeurs manquantes
data = data.dropna()

# --- Fusion HR si manquant ---
if 'HR_Ext' not in data.columns and 'Meteo_pred' in locals():
    data = data.merge(Meteo_pred[['Date', 'HR_Ext']], on='Date', how='left')

# --- Variables explicatives ---
features = [
    'T_Ext', 'HR_Ext', 'Hour', 'Day', 'Month', 'Year', 'Weekend',
    'Puissance_Chaud_lag1', 'Puissance_Chaud_roll6h',
    'Puissance_Froid_lag1', 'Puissance_Froid_roll6h'
]

X = data[features]
y_chaud = data['P_Chaud']
y_froid = data['P_Froid']

# --- Découpage temporel ---
X_train_chaud, X_test_chaud, y_train_chaud, y_test_chaud = train_test_split(X, y_chaud, test_size=0.2, random_state=42, shuffle=False) #shuffle=False pour garder l'ordre temporel  
X_train_froid, X_test_froid, y_train_froid, y_test_froid = train_test_split(X, y_froid, test_size=0.2, random_state=42, shuffle=False)

# --- Modélisation ---
model_chaud = RandomForestRegressor(n_estimators=100, random_state=42) # mettre 200 pour plus de précision
model_chaud.fit(X_train_chaud, y_train_chaud)
y_pred_chaud = model_chaud.predict(X_test_chaud)

model_froid = RandomForestRegressor(n_estimators=100, random_state=42)
model_froid.fit(X_train_froid, y_train_froid)
y_pred_froid = model_froid.predict(X_test_froid)

In [9]:
# --- Évaluation ---
def evaluate_model(y_true, y_pred, label):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    print(f"{label} - MAE: {mae:.2f} MW, RMSE: {rmse:.2f} MW, R²: {r2:.2f}")

evaluate_model(y_test_chaud, y_pred_chaud, "Chaud")
evaluate_model(y_test_froid, y_pred_froid, "Froid")

Chaud - MAE: 1.63 MW, RMSE: 3.07 MW, R²: 0.95
Froid - MAE: 1.04 MW, RMSE: 1.81 MW, R²: 0.97


In [10]:
#--- Visualisation des résultats ---
def plot_results(y_true, y_pred, title,color_real,color_pred):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=y_true.index, y=y_true, mode='lines', name=f"{title} Réel", line=dict(color=color_real)))
    fig.add_trace(go.Scatter(x=y_pred.index, y=y_pred, mode='lines', name=f"{title} Prédit", line=dict(color=color_pred)))
    fig.update_layout(title=title, xaxis_title='Index', yaxis_title='Puissance (MW)')
    fig.update_layout(
        title=f'Comparaison {title} : Réel vs Prédit',
        xaxis_title='Heure (index)',
        yaxis_title='Puissance (MW)',
        legend=dict(x=0.01, y=0.99, bordercolor="gray", borderwidth=0.5),
        template='plotly_white',
        width=950,
        height=450)
    fig.show()

plot_results(y_test_chaud, pd.Series(y_pred_chaud, index=y_test_chaud.index), "Puissance Chaud", 'red', 'orange')
plot_results(y_test_froid, pd.Series(y_pred_froid, index=y_test_froid.index), "Puissance Froid", 'blue', 'lightblue')

In [11]:
# --- Prévisions futures ---
df_future = Meteo_pred.copy()
df_future['Date'] = pd.to_datetime(df_future['Date'])
df_future = df_future.sort_values('Date')

# Création des variables temporelles
df_future['Hour'] = df_future['Date'].dt.hour
df_future['Day'] = df_future['Date'].dt.day
df_future['Month'] = df_future['Date'].dt.month
df_future['Year'] = df_future['Date'].dt.year
df_future['Weekend'] = (df_future['Date'].dt.weekday >= 5).astype(int)
df_future.head()

# --- Séléction des caractéristiques ---
X_future = df_future[['T_Ext', 'HR_Ext', 'Hour', 'Day', 'Month', 'Year', 'Weekend']]

# --- Prévisions futures ---
# Prévisions en 3jours glissants, très fort mais doit tenir compte d'une prévision glissante avec les données récentes 
df_future['P_Chaud_Prédit'] = np.nan
df_future['P_Froid_Prédit'] = np.nan

last_chaud = data['P_Chaud'].iloc[-1]
last_froid = data['P_Froid'].iloc[-1]
last_chaud_roll = data['P_Chaud'].iloc[-6:].mean() # moyenne des 6 dernières heures
last_froid_roll = data['P_Froid'].iloc[-6:].mean()

# Boucle de prévision glissante
for i in range(len(df_future)):
    r=df_future.iloc[i]
    X_row = pd.DataFrame([{
        'T_Ext': r['T_Ext'],
        'HR_Ext': r['HR_Ext'],
        'Hour': r['Hour'],
        'Day': r['Day'],
        'Month': r['Month'],
        'Year': r['Year'],
        'Weekend': r['Weekend'],
        'Puissance_Chaud_lag1': last_chaud,
        'Puissance_Chaud_roll6h': last_chaud_roll,
        'Puissance_Froid_lag1': last_froid,
        'Puissance_Froid_roll6h': last_froid_roll
    }])

    # Prédictions
    pred_chaud = model_chaud.predict(X_row)[0]
    pred_froid = model_froid.predict(X_row)[0]

    #Enregistrement des prédictions
    df_future.at[df_future.index[i], 'P_Chaud_Prédit'] = pred_chaud
    df_future.at[df_future.index[i], 'P_Froid_Prédit'] = pred_froid

    # Mise à jour des valeurs pour l'heure suivante
    last_hot = pred_chaud
    last_cold = pred_froid
    last_hot_roll = pd.concat([data['P_Chaud'].iloc[-5:], df_future['P_Chaud_Prédit'].iloc[:i+1]]).tail(6).mean()
    last_cold_roll = pd.concat([data['P_Froid'].iloc[-5:], df_future['P_Froid_Prédit'].iloc[:i+1]]).tail(6).mean()

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_future['Date'], y=df_future['P_Chaud_Prédit'], mode='lines', name='Puissance Chaud Prédit (MW)', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=df_future['Date'], y=df_future['P_Froid_Prédit'], mode='lines', name='Puissance Froid Prédit (MW)', line=dict(color='lightblue')))
# ajout de la température extérieure pour référence
fig.add_trace(go.Scatter(x=df_future['Date'], y=df_future['T_Ext'], mode='lines', name='Température Extérieure (°C)', line=dict(color='green', dash='dash'), yaxis='y2'))
fig.update_layout(yaxis2=dict(title='Température (°C)', overlaying='y', side='right'))
fig.update_layout(yaxis2=dict(title='Température (°C)', overlaying='y', side='right'))
fig.update_layout(title='Prévisions de Puissance Chaud et Froid, données glissantes', xaxis_title='Date', yaxis_title='Puissance (MW)')
fig.show()

In [53]:
# --- Réentraînement avec variables météo + temporelles ---
features_simple = ['T_Ext', 'HR_Ext', 'Hour', 'Day', 'Month', 'Year', 'Weekend']
X_simple = data[features_simple]
y_chaud = data['P_Chaud']
y_froid = data['P_Froid']

model_chaud_simple = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
model_chaud_simple.fit(X_simple, y_chaud)

model_froid_simple = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
model_froid_simple.fit(X_simple, y_froid)

# --- Préparation des données météo futures ---
df_future = Meteo_pred.copy()
df_future['Date'] = pd.to_datetime(df_future['Date'])
df_future = df_future.sort_values('Date')
df_future['Hour'] = df_future['Date'].dt.hour
df_future['Day'] = df_future['Date'].dt.day
df_future['Month'] = df_future['Date'].dt.month
df_future['Year'] = df_future['Date'].dt.year
df_future['Weekend'] = (df_future['Date'].dt.weekday >= 5).astype(int)

X_future = df_future[['T_Ext', 'HR_Ext', 'Hour', 'Day', 'Month', 'Year', 'Weekend']]

# --- Prédiction directe ---
df_future['P_Chaud_Prédit'] = model_chaud_simple.predict(X_future)
df_future['P_Froid_Prédit'] = model_froid_simple.predict(X_future)

# --- Visualisation ---
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_future['Date'], y=df_future['P_Chaud_Prédit'],
                         mode='lines', name='Puissance Chaud Prévue (MW)', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=df_future['Date'], y=df_future['P_Froid_Prédit'],
                         mode='lines', name='Puissance Froid Prévue (MW)', line=dict(color='deepskyblue')))
fig.update_layout(title='Prévision 72h basée uniquement sur la météo',
                  xaxis_title='Date', yaxis_title='Puissance (MW)',
                  template='plotly_white', width=950, height=450)
fig.show()


Beaucoup plus sensible aux parametres ext et non à l'inertie du réseau.

In [54]:

# params connus
T = range(len(horizon_hours))      # heures à optimiser
I = list(equipements.index)        # indices machines
Pmax = equipements["Pmax_kW"].values / 1000.0  # MW
ENRw = equipements["ENR_weight"].values
dt = 1.0
taux_ENR = Taux_EnR_contractuel     # ex 0.50
E_ENR_prev = enr_prev_mwh           # cumuls 12 mois glissants
E_TOT_prev = tot_prev_mwh

# variables
x = pl.LpVariable.dicts("x",(I,T), lowBound=0, upBound=1)
y = pl.LpVariable.dicts("y",(I,T), lowBound=0, upBound=1, cat="Binary")
s = pl.LpVariable("slack_ENR", lowBound=0)

# objectif (ex: coût)
prob = pl.LpProblem("Dispatch_ENR", pl.LpMinimize)
cost_th = equipements["Cout_th"].values   # €/MWh utile
prob += pl.lpSum(cost_th[i]*Pmax[i]*x[i][t]*dt for i in I for t in T) + 1e6*s

# équilibre demande (faire une pour Chaud, une pour Froid en filtrant I)
# ... tes contraintes de demande ici ...

# bornes Pmin/Pmax
Pmin = equipements["Pmin_kW"].values/1000.0
for i in I:
    for t in T:
        prob += Pmax[i]*x[i][t] <= Pmax[i]*y[i][t]
        prob += Pmax[i]*x[i][t] >= Pmin[i]*y[i][t]

# contrainte ENR sur 12 mois glissants
E_ENR_future = pl.lpSum(ENRw[i]*Pmax[i]*x[i][t]*dt for i in I for t in T)
E_TOT_future = pl.lpSum(Pmax[i]*x[i][t]*dt for i in I for t in T)
prob += E_ENR_prev + E_ENR_future + s >= taux_ENR*(E_TOT_prev + E_TOT_future)

prob.solve(pl.PULP_CBC_CMD(msg=False))


NameError: name 'horizon_hours' is not defined

Dictionnaire des énergies et équipements

In [ ]:
# Dictionnaire des énergies et équipements
Energies_primaires = ['Agropellet', 'Biofioul','Gaz', 'Electricité']

#Verification des pmin et pmax a faire
Informations = {
    "Chaud": {
        "Chaudiere_AL1": {"Pmin_kW": 500, "Pmax_kW": 25000, "Eff": 0.89, "Source": Energies_primaires[1]},
        "Chaudiere_AL2": {"Pmin_kW": 800, "Pmax_kW": 25000, "Eff": 0.89, "Source": Energies_primaires[1]},
        "Chaudiere_AL3": {"Pmin_kW": 600, "Pmax_kW": 26460, "Eff": 0.88, "Source": Energies_primaires[0]},
        "Chaudiere_AL4": {"Pmin_kW": 700, "Pmax_kW": 26460, "Eff": 0.88, "Source": Energies_primaires[0]},
        "Turbine_Gaz": {"Pmin_kW": 1000, "Pmax_kW": 50000, "Eff": 0.92, "Source": Energies_primaires[2]},
        "Chaudiere_Gaz1": {"Pmin_kW": 400, "Pmax_kW": 20000, "Eff": 0.90, "Source": Energies_primaires[2]},
        "Chaudiere_Gaz2": {"Pmin_kW": 600, "Pmax_kW": 20000, "Eff": 0.90, "Source": Energies_primaires[2]},
        "Chaudiere_Gaz3": {"Pmin_kW": 500, "Pmax_kW": 60000, "Eff": 0.90, "Source": Energies_primaires[2]},
        "Chaudiere_Gaz4": {"Pmin_kW": 700, "Pmax_kW": 60000, "Eff": 0.90, "Source": Energies_primaires[2]},
        "TFPc": {"Pmin_kW": 1000, "Pmax_kW": 12000, "Eff": 3.14, "Source": Energies_primaires[3]},
    },
    "Froid": {
        "GF1": {"Pmin_kW": 300, "Pmax_kW": 1500, "Eff": 2.6, "Source": Energies_primaires[3]},
        "GF2": {"Pmin_kW": 400, "Pmax_kW": 1800, "Eff": 3.5, "Source": Energies_primaires[3]},
        "GF3": {"Pmin_kW": 500, "Pmax_kW": 2000, "Eff": 3.3, "Source": Energies_primaires[3]},
        "GF4": {"Pmin_kW": 600, "Pmax_kW": 2200, "Eff": 3.4, "Source": Energies_primaires[3]},
        "GF5": {"Pmin_kW": 700, "Pmax_kW": 2500, "Eff": 3.6, "Source": Energies_primaires[3]},
        "GF6": {"Pmin_kW": 800, "Pmax_kW": 2800, "Eff": 3.5, "Source": Energies_primaires[3]},
        "GF7": {"Pmin_kW": 900, "Pmax_kW": 3000, "Eff": 3.7, "Source": Energies_primaires[3]},
        "TFPf": {"Pmin_kW": 1200, "Pmax_kW": 3500, "Eff": 3.14, "Source": Energies_primaires[3]},
    }
}

rows = []
for service,dic in Informations.items():
    for equipement, details in dic.items():
        rows.append({"Equipement": equipement, "Service": service, **details})

equipements_df = pd.DataFrame(rows)
print(equipements_df)

prix_energies = {
    'Agropellet': 40,  # €/MWh # Prix moyen en France, peut varier, mettre a jour
    'Biofioul': 80,    # €/MWh # Prix moyen en France, peut varier, mettre a jour
    'Gaz': 75,         # €/MWh # Prix moyen en France, peut varier, mettre a jour
    'Electricité': 150 # €/MWh # Prix moyen en France, peut varier, mettre a jour, voir pour mettre variable grace API
}
ef = {'Agropellet': 0.025, 'Biofioul': 0.27, 'Gaz': 0.202, 'Electricité': 0.053}  # tCO2/MWh a definir pour chaque energie, valeur a changer.
enr = {'Agropellet': 1, 'Biofioul': 1, 'Gaz': 0, 'Electricité': 0.72} # part renouvelable a definir pour chaque energie

equipements_df['Prix_€/MWh'] = equipements_df['Source'].map(prix_energies)
equipements_df['Emissions_tCO2/MWh'] = equipements_df['Source'].map(ef)
equipements_df['Part_renouvelable'] = equipements_df['Source'].map(enr)

print(equipements_df)

# Coût/CO2 thermique (sortie utile) selon service
def cout_th(row):
    if row['Service'] == 'Chaud' and row['Source'] != 'Electricité':
        return row['Prix_€/MWh'] / row['Eff']
    elif row['Service'] == 'Froid' and row['Source'] != 'Electricité':
        return row['Prix_€/MWh'] / row['Eff']
    else:
        return prix_energies['Electricité']/row['Eff']
    
def co2_th(row):
    if row['Service'] == 'Chaud' and row['Source'] != 'Electricité':
        return row['Emissions_tCO2/MWh'] / row['Eff']
    elif row['Service'] == 'Froid' and row['Source'] != 'Electricité':
        return row['Emissions_tCO2/MWh'] / row['Eff']
    else:
        return ef['Electricité']/row['Eff']
    
equipements_df['Coût_thermique_€/MWh'] = equipements_df.apply(cout_th, axis=1)
equipements_df['CO2_thermique_tCO2/MWh'] = equipements_df.apply(co2_th, axis=1)

        Equipement Service  Pmin_kW  Pmax_kW   Eff       Source
0    Chaudiere_AL1   Chaud      500    25000  0.89     Biofioul
1    Chaudiere_AL2   Chaud      800    25000  0.89     Biofioul
2    Chaudiere_AL3   Chaud      600    26460  0.88   Agropellet
3    Chaudiere_AL4   Chaud      700    26460  0.88   Agropellet
4      Turbine_Gaz   Chaud     1000    50000  0.92          Gaz
5   Chaudiere_Gaz1   Chaud      400    20000  0.90          Gaz
6   Chaudiere_Gaz2   Chaud      600    20000  0.90          Gaz
7   Chaudiere_Gaz3   Chaud      500    60000  0.90          Gaz
8   Chaudiere_Gaz4   Chaud      700    60000  0.90          Gaz
9             TFPc   Chaud     1000    12000  3.14  Electricité
10             GF1   Froid      300     1500  2.60  Electricité
11             GF2   Froid      400     1800  3.50  Electricité
12             GF3   Froid      500     2000  3.30  Electricité
13             GF4   Froid      600     2200  3.40  Electricité
14             GF5   Froid      700     